# 반복 평가

실험에 반복을 추가할 수 있습니다.

이는 평가를 여러 번 반복할 수 있게 하여 다음과 같은 경우에 유용합니다:

- 더 큰 평가 세트의 경우
- 가변적인 응답을 생성할 수 있는 체인의 경우
- 가변적인 점수를 생성할 수 있는 평가(예: llm-as-judge)의 경우

https://docs.smith.langchain.com/how_to_guides/evaluation/evaluate_llm_application#evaluate-on-a-dataset-with-repetitions

In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


In [3]:
from myrag import PDFRAG


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        # 질문에 대한 컨텍스트 검색
        context = retriever.invoke(inputs["question"])
        # 검색된 문서들을 하나의 문자열로 결합
        context = "\n".join([doc.page_content for doc in context])
        # 질문, 컨텍스트, 답변을 포함한 딕셔너리 반환
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question

In [4]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama


gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=1.0))

# Ollama 모델을 불러옵니다.
ollama_chain = ask_question_with_llm(
    ChatOllama(model="EEVE-Korean-10.8B:latest", temperature=1.0)
)

### GPT 모델을 활용한 RAG 에 대하여 반복 평가

In [7]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
cot_qa_evalulator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4o-mini", temperature=0)},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"],
    },
)

dataset_name = "RAG_EVAL_DATASET_teddynote"

# 평가 실행
evaluate(
    gpt_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="REPEAT_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Repeat 평가를 수행합니다. GPT-4o-mini 모델 (cot_qa)",
    },
    num_repetitions=3,
)

View the evaluation results for experiment: 'REPEAT_EVAL-d7266763' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=7546d053-dc69-4d80-9e26-87225ae62232




0it [00:00, ?it/s]

Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running evaluator <DynamicRunEvaluator evaluate> on run 7b7528ca-8d2a-4f9a-a888-a31a941b85b0: KeyError('answer')
Traceback (most recent call last):
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\_runner.py", line 1345, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Us

<ExperimentResults REPEAT_EVAL-d7266763>

### Ollama 모델을 활용한 RAG 에 대하여 반복 평가

In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
cot_qa_evalulator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4o-mini", temperature=0)},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"],
    },
)

dataset_name = "RAG_EVAL_DATASET_teddynote"

# 평가 실행
evaluate(
    ollama_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="REPEAT_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Repeat 평가를 수행합니다. EEVE-Korean-10.8B 모델 (cot_qa)",
    },
    num_repetitions=3,
)

View the evaluation results for experiment: 'REPEAT_EVAL-ee9360c0' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=a5b2a09f-6289-42aa-b40e-f897bbbd9076




0it [00:00, ?it/s]

Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running evaluator <DynamicRunEvaluator evaluate> on run 73b3e8b4-2549-48fa-b5f0-a70935767938: KeyError('answer')
Traceback (most recent call last):
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\_runner.py", line 1345, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\vir